In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai faiss-cpu

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./assets-resources/tree-of-thoughts.pdf")
documents = loader.load()

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

retriever = db.as_retriever()

from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "tree_of_thoughts",
    "Searches and returns excerpts from the tree of thoughts paper.",
)
tools = [tool]

from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

result = agent_executor.invoke({"input": "hi, im bob"})

result["output"]


result = agent_executor.invoke(
    {
        "input": "what is the tree of thoughts technique?"
    }
)

result["output"]

'The Tree of Thoughts technique is a framework for language model inference that allows models to perform deliberate decision-making by considering multiple reasoning paths and self-evaluating choices to decide the next course of action. It generalizes over the popular "Chain of Thought" approach and enables exploration over coherent units of text ("thoughts") that serve as intermediate steps toward problem-solving. The Tree of Thoughts framework allows language models to explore multiple reasoning paths over thoughts, framing any problem as a search over a tree where each node represents a partial solution with the input and the sequence of thoughts so far.\n\nThis technique addresses the shortcomings of existing approaches by incorporating exploration of different continuations within a thought process locally and incorporating planning, lookahead, or backtracking globally to evaluate different options. By decomposing the intermediate process into thought steps, generating potential 

In [5]:
result = agent_executor.invoke(
    {"input": "How did they evaluated the performance of the technique?"}
)

In [6]:
result["output"]

'The performance of the technique was evaluated through various tasks such as the Game of 24, Creative Writing, and Mini Crosswords. The Tree of Thoughts (ToT) framework was used for problem-solving with language models. The ToT framework actively maintains a tree of thoughts, where each thought is a coherent language sequence serving as an intermediate step towards problem-solving. The LM self-evaluates the progress of different intermediate thoughts through a deliberate reasoning process instantiated in language. By combining language-based capability to generate and evaluate diverse thoughts with search algorithms like breadth-first search (BFS) or depth-first search (DFS), systematic exploration of the tree of thoughts with lookahead and backtracking was achieved. The ToT framework showed superior results in all three tasks compared to existing LM inference methods, even with the state-of-the-art language model, GPT-4. The tasks required deductive, mathematical, commonsense, and le